# Prediction Health care Professionals from add data



In [ ]:
!pip install tensorflow tensorflow_decision_forests --quiet
!pip install catboost --quiet
!pip install xgboost --quiet
!pip install category_encoders --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2, _, _, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import chi2_contingency
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from urllib.parse import urlparse
from google.colab import drive
from zipfile import ZipFile
import xgboost as xgb
import pandas as pd
import numpy as np
import time
import math
import re

In [ ]:
class progressbar:
    def __init__(self,timegap,length=40,char1='-',char2='>'):
        self.start_time = time.time()
        self.timegap=timegap
        self.time_ellapsed=0
        self.length = length
        self.char1 = char1
        self.char2 = char2
        self.data=[]
        self.extraword=''
        self.past = 0
        self.tim1 = 0
        self.past_error = 0
    def print(self,current,max,string='',clear=True,graph=False,error=0,graph_length=10,smoothing=0.2):
        assert smoothing<1
        progress=round(self.length*current/max)
        if progress == 0:
            self.past_error = error
        if time.time()-self.start_time>self.timegap:
            self.start_time = time.time()
            self.time_ellapsed+=self.timegap
            self.extraword+='\n Time taken ='+str(self.time_ellapsed)+'sec'
            if clear:
                clear_output(wait=True)
                if progress!=self.past:
                    self.tim1 = round((self.length-progress)/(progress-self.past+0.1)*self.timegap,2)
                print('['+self.char1*int(progress)+self.char2+' '*(self.length-progress)+']',
                        '\n',
                        string,
                        f"\ntime {self.tim1}s",
                        f"\ntime_spent {self.time_ellapsed}s",
                        flush=True)
                self.past = progress
            if graph:
                if len(self.data)>graph_length:
                    self.data.pop(0)
                    # print(self.data,graph_length,len(self.data))
                error = smoothing*error+(1-smoothing)*self.past_error
                self.past_error = error
                self.data.append([self.time_ellapsed,error])
                plt.plot([x[0] for x in self.data],[x[1] for x in self.data])
                plt.show()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# with ZipFile('/content/drive/MyDrive/Hackathon Testbiz/Correct-Complete-DataSet.zip','r') as f:
#     f.extractall()

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Hackathon Testbiz/Doceree-HCP_Train.csv",encoding='ISO-8859-1')

In [ ]:
df.isna().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [ ]:
df = df.drop(['ID','CHANNELTYPE'],axis = 1)

In [ ]:
df.head(2)

,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0


In [ ]:
df = df[df.IS_HCP.isna()==0].reset_index().drop('index',axis=1)

In [ ]:
train_df,test_df,train_y,test_y = train_test_split(df.drop('IS_HCP',axis=1),df.IS_HCP,test_size=0.2,random_state=42)

In [ ]:
url_counts = train_df.URL.value_counts()
key_counts = train_df.KEYWORDS.value_counts()
bdr_counts = train_df.BIDREQUESTIP.value_counts()

In [ ]:
!wget https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip

--2023-07-02 07:44:17--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip
Resolving huggingface.co (huggingface.co)... 13.35.166.50, 13.35.166.114, 13.35.166.69, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.50|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/stanfordnlp/glove/6471382cdd837544bf3ac72497a38715e845897d265b2b424b4761832009c837?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27glove.6B.zip%3B+filename%3D%22glove.6B.zip%22%3B&response-content-type=application%2Fzip&Expires=1688540356&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3N0YW5mb3JkbmxwL2dsb3ZlLzY0NzEzODJjZGQ4Mzc1NDRiZjNhYzcyNDk3YTM4NzE1ZTg0NTg5N2QyNjViMmI0MjRiNDc2MTgzMjAwOWM4Mzc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjg4NTQwMzU2fX19XX0_&Signature=i0Yb0Z%7EVOm3j-2wkgDErpVjl

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/glove.6B.zip','r') as f:
    f.extractall()

In [ ]:
with open('/content/glove.6B.300d.txt','r') as f:
    word_to_vec = {}
    for i in f.readlines():
        i = i.strip('\n').split()
        word_to_vec[i[0]] = np.array(list(map(float,i[1:])))

In [ ]:
l = set()
def func(x):
    x = set(x.lower().split('|'))
    k = []
    for i in x:
        i = i.replace('-','')
        if len(i.split())>1:
            for i in i.split():
                if i in word_to_vec:
                    k.append(word_to_vec[i])
                else:
                    l.add(i)
        if i in word_to_vec:
            k.append(word_to_vec[i])
        else:
            l.add(i)
    return pd.Series(sum(k)/len(k))
keys = train_df.KEYWORDS.apply(func)

In [ ]:
train_keys = keys
test_keys = test_df.KEYWORDS.apply(func)

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_keys,train_y)
y_pred = clf.predict(test_keys)
print(f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,clf.predict(train_keys),average = 'macro'))

0.9883267951298134 0.9899830689432558


In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_keys,train_y)
y_pred = clf.predict(test_keys)
print(f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,clf.predict(train_keys),average = 'macro'))

0.9883267951298134 0.9899830689432558


In [ ]:
import category_encoders as ce
encoder1 = ce.TargetEncoder(cols=['BIDREQUESTIP'],min_samples_leaf=20,smoothing=1)
encoder2 = ce.TargetEncoder(cols=['USERPLATFORMUID'],min_samples_leaf=20,smoothing=1)
encoder3 = ce.TargetEncoder(cols=['USERAGENT'],min_samples_leaf=20,smoothing=1)
encoder4 = ce.TargetEncoder(cols=['USERZIPCODE'],min_samples_leaf=30,smoothing=1)
encoder5 = ce.TargetEncoder(cols=['URL'],min_samples_leaf=20,smoothing=1.1)  #good val 1.09
encoder6 = ce.TargetEncoder(cols=['USERCITY'],min_samples_leaf=24,smoothing=1.35)
# encoder7 = ce.TargetEncoder(cols=['PLATFORMTYPE'],min_samples_leaf=20,smoothing=1.35)
# PLATFORMTYPE
x_tr_encoded = encoder1.fit_transform(train_df, train_y)
x_te_encoded = encoder1.transform(test_df)

x_tr_encoded = encoder2.fit_transform(x_tr_encoded, train_y)
x_te_encoded = encoder2.transform(x_te_encoded)

x_tr_encoded = encoder3.fit_transform(x_tr_encoded, train_y)
x_te_encoded = encoder3.transform(x_te_encoded)

x_tr_encoded = encoder4.fit_transform(x_tr_encoded, train_y)
x_te_encoded = encoder4.transform(x_te_encoded)

x_tr_encoded = encoder5.fit_transform(x_tr_encoded, train_y)
x_te_encoded = encoder5.transform(x_te_encoded)

x_tr_encoded = encoder6.fit_transform(x_tr_encoded, train_y)
x_te_encoded = encoder6.transform(x_te_encoded)

# x_tr_encoded = encoder7.fit_transform(x_tr_encoded, train_y)
# x_te_encoded = encoder7.transform(x_te_encoded)

In [ ]:
dim = 200
train_keys.columns = [f'{x}_' for x in range(dim)]
test_keys.columns = [f'{x}_' for x in range(dim)]

In [ ]:
train_data = pd.concat([x_tr_encoded[['BIDREQUESTIP','USERCITY','USERPLATFORMUID','USERAGENT','USERZIPCODE','URL']],
                        train_keys],axis=1)
test_data = pd.concat([x_te_encoded[['BIDREQUESTIP','USERCITY','USERPLATFORMUID','USERAGENT','USERZIPCODE','URL']],
                        test_keys],axis=1)

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_data,train_y)
y_pred = clf.predict(test_data)
test_score = f1_score(test_y,y_pred,average = 'macro')
train_score = f1_score(train_y,clf.predict(train_data),average = 'macro')
test_score,train_score

(0.9963575212167836, 1.0)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf = ExtraTreesClassifier(n_estimators=160, n_jobs=-1,random_state=0)

clf.fit(train_data, train_y)

y_pred = clf.predict(test_data)
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,clf.predict(train_data),average = 'macro')

(0.9985428884261818, 1.0)

In [ ]:
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,clf.predict(train_data),average = 'macro')

(0.9982307960195806, 1.0)

In [ ]:
import time
bt =time.time()
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0,n_estimators=160)
RFC.fit(train_data, train_y)
y_pred = RFC.predict(test_data)
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,RFC.predict(train_data),average = 'macro'),time.time()-bt

(0.9980747690887307, 1.0)

In [ ]:
pd.Series(RFC.feature_importances_,index = train_data.columns)[:10]

BIDREQUESTIP       0.045040
USERCITY           0.011290
USERPLATFORMUID    0.046247
USERAGENT          0.100293
USERZIPCODE        0.030142
URL                0.289503
0_                 0.001362
1_                 0.001460
2_                 0.001912
3_                 0.008588
dtype: float64

In [ ]:
pd.Series(clf.feature_importances_,index = train_data.columns)[:10]

BIDREQUESTIP       0.025542
USERCITY           0.009104
USERPLATFORMUID    0.023108
USERAGENT          0.140809
USERZIPCODE        0.022761
URL                0.276180
0_                 0.002437
1_                 0.002113
2_                 0.001968
3_                 0.003877
dtype: float64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0,n_estimators=165)
RFC.fit(train_data, train_y)
y_pred = RFC.predict(test_data)
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,RFC.predict(train_data),average = 'macro')

(0.9982827606876497, 0.9999346326510605)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0,n_estimators=100)
RFC.fit(train_data, train_y)
y_pred = RFC.predict(test_data)
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,RFC.predict(train_data),average = 'macro')

(0.9980747690887307, 0.9999477055725281)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0,n_estimators=170)
RFC.fit(train_data, train_y)
y_pred = RFC.predict(test_data)
f1_score(test_y,y_pred,average = 'macro'),f1_score(train_y,RFC.predict(train_data),average = 'macro')

(0.9983869686959636, 0.9999346326510605)

In [ ]:
final_test_data = pd.read_csv('/content/drive/MyDrive/Hackathon Testbiz/Doceree-HCP_Test.csv')

In [ ]:
final_test_keys = final_test_data.KEYWORDS.apply(func)

In [ ]:
id = final_test_data.ID
final_test_data = final_test_data.drop('ID',axis=1)
x_te_encoded = encoder1.transform(final_test_data)
x_te_encoded = encoder2.transform(x_te_encoded)
x_te_encoded = encoder3.transform(x_te_encoded)
x_te_encoded = encoder4.transform(x_te_encoded)
x_te_encoded = encoder5.transform(x_te_encoded)
x_te_encoded = encoder6.transform(x_te_encoded)
final_test_keys.columns = [f'{x}_' for x in range(dim)]
final_test_data  = pd.concat([x_te_encoded[['BIDREQUESTIP','USERCITY','USERPLATFORMUID','USERAGENT','USERZIPCODE','URL']],
                        final_test_keys],axis=1)

In [ ]:
final_train_keys = df.KEYWORDS.apply(func)

In [ ]:
x_te_encoded = encoder1.transform(df.drop('IS_HCP',axis=1))
x_te_encoded = encoder2.transform(x_te_encoded)
x_te_encoded = encoder3.transform(x_te_encoded)
x_te_encoded = encoder4.transform(x_te_encoded)
x_te_encoded = encoder5.transform(x_te_encoded)
x_te_encoded = encoder6.transform(x_te_encoded)
final_train_keys.columns = [f'{x}_' for x in range(dim)]
training  = pd.concat([x_te_encoded[['BIDREQUESTIP','USERCITY','USERPLATFORMUID','USERAGENT','USERZIPCODE','URL']],
                        final_train_keys],axis=1)

In [ ]:
training.head(2)

,BIDREQUESTIP,USERCITY,USERPLATFORMUID,USERAGENT,USERZIPCODE,URL,0_,1_,2_,3_,...,190_,191_,192_,193_,194_,195_,196_,197_,198_,199_
0,0.219795,0.263014,0.299489,0.042780,0.227273,0.00156,-0.377835,0.169633,0.531444,-0.087512,...,0.327756,0.126030,0.372744,0.142137,-0.301632,0.158243,-0.119097,-0.098144,0.016123,-0.124896
1,0.000000,0.038811,0.299491,0.159299,0.002982,0.00000,-0.291496,0.047117,0.354221,0.062432,...,0.298001,0.175415,0.319247,0.047611,-0.283282,0.003694,-0.010721,0.175156,-0.073016,-0.047308


In [ ]:
ERFC = ExtraTreesClassifier(n_estimators=175, n_jobs=-1,random_state=0)
ERFC.fit(training, df.IS_HCP)

ExtraTreesClassifier(n_estimators=175, n_jobs=-1, random_state=0)

In [ ]:
y_pred = ERFC.predict(final_test_data)

In [ ]:
final_test_data['IS_HCP'] = y_pred

In [ ]:
final_test_data['IS_HCP'] = final_test_data['IS_HCP'].apply(lambda x:int(x))

In [ ]:
final_test_data['ID'] = id

In [ ]:
final_test_data[['ID','IS_HCP']].set_index('ID').to_csv('/content/NBR_Submission_45.csv')

In [ ]:
f1_score(df.IS_HCP,ERFC.predict(training),average='macro')

1.0

In [ ]:
final_test_data.IS_HCP.value_counts() # 99.8571

0    20032
1     8461
Name: IS_HCP, dtype: int64

In [ ]:
test_data.IS_HCP.value_counts()

0    20040
1     8453
Name: IS_HCP, dtype: int64

In [ ]:
test_data.IS_HCP.value_counts()

0    20039
1     8454
Name: IS_HCP, dtype: int64